In [3]:
import pandas as pd
import numpy as np
import torch
import os

from transformers import AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from customdataset import ConfidenceDataset, TestDataset
from datasets import load_metric, load_dataset
from custommodel import RobertaForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from utils import set_allseed
import warnings
import pickle

In [4]:
seed = 777
batch_size = 32
save_steps = 103
set_allseed(seed)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
gpu = torch.device("cuda")
cpu = torch.device("cpu")
warnings.filterwarnings(action='ignore') 

In [5]:
def compute_metrics(pred):
    f1 = load_metric("f1")
    references = pred.label_ids
    predictions = pred.predictions.argmax(axis=1)
    metric = f1.compute(predictions=predictions, references=references, average="micro")
    return metric

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [7]:
kfold_function = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
config = AutoConfig.from_pretrained("klue/roberta-base")
config.num_labels = len(train_df.확실성.value_counts())

In [8]:
training_args = TrainingArguments(
    output_dir="./output_confidence",
    seed=seed,
    save_total_limit=2,
    save_steps = save_steps,
    num_train_epochs = 5,
    learning_rate= 1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    weight_decay=1e-4,
    logging_dir="./logs",
    logging_steps = save_steps,
    evaluation_strategy = "steps",
    eval_steps = save_steps,
    load_best_model_at_end=True,
)
test_dataset = TestDataset(data=test_df, tokenizer=tokenizer)

In [9]:
logit = 0
for i, (train_index, test_index) in enumerate(kfold_function.split(train_df["확실성"],train_df["확실성"])):
    model = RobertaForSequenceClassification.from_pretrained("klue/roberta-base", config=config)
    train_corpus, valid_corpus = train_df["문장"][train_index], train_df["문장"][test_index]
    train_label, valod_label = train_df["확실성"][train_index], train_df["확실성"][test_index]
    fold_train = pd.concat([train_corpus, train_label], axis =1)
    fold_valid = pd.concat([valid_corpus, valod_label], axis =1)
    train_dataset = ConfidenceDataset(data=fold_train, tokenizer=tokenizer)
    valid_dataset = ConfidenceDataset(data=fold_valid, tokenizer=tokenizer)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    )

    trainer.train()

    logit += trainer.predict(test_dataset).predictions / 5

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['lstm.weight_hh_l0_reverse', 'roberta.pooler.dense.weight', 'ls

  0%|          | 0/515 [00:00<?, ?it/s]

In [ ]:
with open("confidence_logit_focal.pickle","wb") as f:
    pickle.dump(logit, f)

In [ ]:
result = pd.DataFrame(logit.argmax(axis=1).tolist(), columns=["confidence"])
test_df_result = pd.concat([test_df,result],axis=1, ignore_index=True)
test_df_result.to_csv("result_confidence.csv")